In [1]:
import SimpleITK as sitk
import os
import numpy as np

In [2]:
# Related notebooks
# Create ventricular landmarks (the matching annotation in the CCF)

In [3]:
# input_directory
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/"

In [4]:
# output directory
output_directory = 'landmarks_v06/fmost_annotation'
os.makedirs(output_directory,exist_ok=True)

In [5]:
def overlay( base, image ) :
    
    barr = sitk.GetArrayFromImage( base )
    iarr = sitk.GetArrayFromImage( image )
    
    idx = np.where( iarr > 0 )
    barr[idx] = iarr[idx]
    
    output_image = sitk.GetImageFromArray( barr )
    output_image.CopyInformation( base )    
    return output_image

In [6]:
# list of annotations to be processed
adict = {}

In [7]:
a = {}
a['annotation_file'] = os.path.join( model_directory,  "Quanxin", "fimbra_habenular_choriod_plexus_optic_tract_20010609.nrrd")
lut = {}
lut[1] = 6
lut[4] = 11
lut[2] = 8
lut[5] = 9
lut[6] = 12
a['lut'] = lut
a['flip'] = False
adict['fimbra_habenular_choriod_plexus_optic_tract'] = a

In [8]:
a = {}
a['annotation_file'] = os.path.join( model_directory,  "Quanxin", "CP-fMOST9272021.nrrd")
lut = {}
lut[2] = 13
a['lut'] = lut
a['flip'] = True
adict['CP'] = a

In [9]:
a = {}
a['annotation_file'] = os.path.join( model_directory,  "Quanxin", "CA-DG-SUB_modified.nrrd")
lut = {}
lut[2] = 14
a['lut'] = lut
a['flip'] = True
adict['HIP'] = a

In [10]:
combined = None

for a in adict :
    
    print("processing:" + a)
    print(" : " + adict[a]['annotation_file'])
    
    # open volume and convert to array
    annotation = sitk.ReadImage( adict[a]['annotation_file'] )
    
    if adict[a]['flip'] :
        print(" : " + "flipping L/R")
        annotation = sitk.Flip( annotation, [True,False,False])
        
    input_array = sitk.GetArrayFromImage( annotation )
    
    # initialized output array
    output_array = np.zeros( input_array.shape, input_array.dtype )
    
    # renumber the segementation mask
    lut = adict[a]['lut']
    for k in lut :
        idx = np.where( input_array == k )
        output_array[idx] = lut[k]
        
    renumbered = sitk.GetImageFromArray( output_array )
    renumbered.CopyInformation( annotation )
    
    if combined == None :
        combined = overlay( renumbered, renumbered )
    else :
        combined = overlay(combined, renumbered)

    

processing:fimbra_habenular_choriod_plexus_optic_tract
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/Quanxin\fimbra_habenular_choriod_plexus_optic_tract_20010609.nrrd
processing:CP
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/Quanxin\CP-fMOST9272021.nrrd
 : flipping L/R
processing:HIP
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/Quanxin\CA-DG-SUB_modified.nrrd
 : flipping L/R


In [11]:
output_file = os.path.join( output_directory, 'fmost_landmarks_one_sided.nrrd')
sitk.WriteImage( combined, output_file, True )

In [12]:
# Next: create double sided version of the landmarks